In [1]:
from vectorhash_imported import *
from vectorhash_convered import *
from nd_scaffold import GridScaffold
import math
from scipy.stats import norm


lambdas = [2,3]
shapes = [(i, i) for i in lambdas]
percent_nonzero_relu = 0.8
W_gh_var = 0.1
sparse_initialization = 0.1
T = 0.01 
W_hg_std = math.sqrt(W_gh_var)
W_hg_mean = -W_hg_std * norm.ppf(1 - percent_nonzero_relu) / math.sqrt(len(lambdas))
h_normal_mean = len(lambdas) * W_hg_mean
h_normal_std = math.sqrt(len(lambdas)) * W_hg_std
relu_theta = math.sqrt((1 - sparse_initialization) * len(lambdas)) * norm.ppf(
    1 - percent_nonzero_relu
)
num_imgs = 5

print(percent_nonzero_relu, W_hg_mean, W_hg_std, h_normal_mean, h_normal_std, relu_theta)

GS = GridScaffold(
    shapes=shapes,
    N_h=1200,
    input_size=5,
    h_normal_mean=h_normal_mean,
    h_normal_std=h_normal_std,
    device=None,
    sparse_matrix_initializer=SparseMatrixBySparsityInitializer(
        sparsity=sparse_initialization, device="cpu"
    ),
    relu_theta=relu_theta, ######
    from_checkpoint=False,
    T=T,
    continualupdate=False,
    ratshift=False,
    initialize_W_gh_with_zeroes=False,
    pseudo_inverse=False,
    batch_update=False,
    use_h_fix=False,
    learned_pseudo=True,
    epsilon=0.01,
    calculate_update_scaling_method="n_h",
    MagicMath=False,
    sanity_check=True,
    calculate_g_method="fast",
)

0.8 0.18819222895762647 0.31622776601683794 0.37638445791525293 0.447213595499958 -1.1291533737457589
module shapes:  [(2, 2), (3, 3)]
N_g     :  13
N_patts :  36
N_h     :  1200


In [2]:
def prepare_data(
    dataset,
    num_imgs=5,
    preprocess_sensory=True,
    noise_level="medium",
    use_fix=False,
):
    import torch
    import random

    data = dataset.data
    # print(num_imgs)
    # print(data.shape)
    data = data.reshape(data.shape[0], -1)
    # print(data.shape)
    data = torch.tensor(data[:num_imgs]).float().to("cpu")
    # print(data.shape)

    # data = random.sample(dataset.data.flatten(1).float().to("cpu"), num_imgs)
    if preprocess_sensory:
        if use_fix:
            for i in range(len(data)):
                data[i] = (data[i] - data[i].mean()) / data[i].std()

        data = (data - data.mean()) / data.std()
    # noissing the data
    if noise_level == "none":
        return data, data
    elif noise_level == "low":
        random_noise = torch.zeros_like(data).uniform_(-1, 1)
    elif noise_level == "medium":
        random_noise = torch.zeros_like(data).uniform_(-1.25, 1.25)
    elif noise_level == "high":
        random_noise = torch.zeros_like(data).uniform_(-1.5, 1.5)
    noisy_data = data + random_noise

    return data, noisy_data

def prepare_data_random(noise_scale=0.1):
    data = torch.randn((num_imgs, 5))
    noise = noise_scale * torch.randn((num_imgs, 5))
    return data, data + noise

In [3]:
from vectorhash_functions import solve_mean, spacefillingcurve
import torchvision
from torchvision import transforms
import torch

transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Lambda(lambda x: x.flatten())]
)

dataset = torchvision.datasets.MNIST(
    root="data", train=True, download=True, transform=transform
)

# data, noisy_data = prepare_data(
#     dataset,
#     num_imgs=num_imgs,
#     preprocess_sensory=True,
#     noise_level="none",
#     use_fix=True
# )
data, noisy_data =prepare_data_random(noise_scale=0.1) 

v = spacefillingcurve(shapes)

g_points, g_points2 = GS.learn_path(observations=data, velocities=v[: len(data)])
recalled_imgs = GS.recall(noisy_data)
similarity = torch.cosine_similarity(data, recalled_imgs, dim=1)
print(similarity.mean())


/home/ezrahuang/miniconda3/envs/ml/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/ezrahuang/miniconda3/envs/ml/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/home/ezrahuang/Projects/rl-research/vectorhash/nd_scaffold.py:548: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3725.)
  1 + input.T @ self.inh

AssertionError: Whs should be the pseudo-inverse of Wsh. Got tensor([0.0000, 0.0000, 0.0155,  ..., 0.0060, 0.0072, 0.0071]) and expected tensor([0.0000, 0.0000, 3.4812,  ..., 1.3494, 1.6256, 1.6081])

In [ ]:
from vectorhash_functions import *

mod_n_positions = []
mod_n_positions_2 = []

# assume shapes are squares

for _ in lambdas:
    mod_n_positions.append(list())
    mod_n_positions_2.append(list())

for g in g_points:
    pos = 0
    for i, l in enumerate(lambdas):
        mod_n = g[pos:pos + l**2]
        mod_n_positions[i].append(ConvertToXYNew(mod_n, (l,l)))
        pos += l**2

for g in g_points2:
    pos = 0
    for i, l in  enumerate(lambdas):
        mod_n = g[pos:pos + l**2]
        mod_n_positions_2[i].append(ConvertToXYNew(mod_n, (l,l)))
        pos += l**2

In [ ]:
for i, l in enumerate(lambdas):
  GraphGrid((l,l), mod_n_positions[i], first_point=mod_n_positions[i][0], title=f"estimated {l}x{l} grid position of first image over time. \nN_h={GS.N_h}, sparsity={sparse_initialization}, relu_theta={relu_theta}, W_hg_mean={W_hg_mean}, W_hg_std={W_hg_std}\n shapes={shapes}, num_imgs={num_imgs}")
  GraphGrid((l,l), mod_n_positions_2[i], first_point=mod_n_positions_2[i][0], title=f"estimated {l}x{l} grid position of second image over time. \nN_h={GS.N_h}, sparsity={sparse_initialization}, relu_theta={relu_theta}, W_hg_mean={W_hg_mean}, W_hg_std={W_hg_std}\n shapes={shapes}, num_imgs={num_imgs}")

In [ ]:
# graph estimated position of each image

In [ ]:
print(data.shape)

In [ ]:
point_positions_n = []
for l in lambdas:
    point_positions_n.append([])

for img in data:
    pos = GS.estimate_position(img)

    p=0
    for i, l in enumerate(lambdas):
        mod_n = pos[p:p+l**2]
        point_positions_n[i].append(ConvertToXYNew(mod_n, (l,l)))
        p += l**2

for i, l in enumerate(lambdas):
    GraphGrid((l,l), point_positions_n[i], title=f"estimated {l}x{l} grid position of each image. \nN_h={GS.N_h}, sparsity={sparse_initialization}, relu_theta={relu_theta}, W_hg_mean={W_hg_mean}, W_hg_std={W_hg_std}\n shapes={shapes}, num_imgs={num_imgs}")

In [ ]:
img = data[1]
recalled = GS.recall(img)
print(torch.cosine_similarity(img, recalled, dim=0).mean())

plt.imshow(img.reshape(5,1))
plt.show()
plt.imshow(recalled.reshape(5,1))
plt.show()
